# Smart Travel App
## Notebook 4: Machine Learning

**Group 4, Project 4**  
Team: Dominique Villarreal, Enrique Garcia, Jose Santos  
Project Due Date: June 12, 2023


In [1]:
import json
import os
import pandas as pd
import pprint as pp
import random
import requests
import time
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# Import csv's & load df's
# Denver
#merged_file_path = '/Users/josesantos/LearnPython/BCGit/G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_file_path = '../G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_df = pd.read_csv(merged_file_path)

merged_df.head(2)

,categories,name,rating,review_count,location,coordinates,city
0,"['Festivals', 'Arts & Crafts', 'Local Flavor']",Jackalope Arts,5.0,8,"{'address1': '5738 Olde Wadsworth Blvd', 'addr...","{'latitude': 39.800724136818026, 'longitude': ...",Denver
1,"['Venues & Event Spaces', 'Party & Event Plann...",Special Occasions Events,5.0,4,"{'address1': '3550 Federal Blvd', 'address2': ...","{'latitude': 39.76623, 'longitude': -105.02439}",Denver


In [3]:
# Process & Organize merged_df
columns_to_delete = ['location', 'coordinates']
merged_df.drop(columns_to_delete, axis=1, inplace=True)

merged_df = merged_df[['city','name', 'rating', 
                       'review_count', 'categories']]

# Access and work with the updated DataFrame
merged_df.head(1)

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"


In [93]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19627 entries, 0 to 19626
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          19627 non-null  object 
 1   name          19627 non-null  object 
 2   rating        19627 non-null  float64
 3   review_count  19627 non-null  int64  
 4   categories    19627 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 766.8+ KB


In [5]:
# Begin work for predicting city

# Create encoded df
city_encoded_data = merged_df.copy()
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"
1,Denver,Special Occasions Events,5.0,4,"['Venues & Event Spaces', 'Party & Event Plann..."
2,Denver,Denver International Wine Festival,4.5,6,['Festivals']
3,Denver,"Denver Mineral, Fossil, Gem & Jewelry Show",4.5,10,['Festivals']
4,Denver,Colorado Festival of Horror,5.0,1,['Festivals']
...,...,...,...,...,...
19622,NewYork,Felice 56,4.0,94,"['Italian', 'Wine Bars']"
19623,NewYork,The Jeffrey Craft Beer & Bites,4.0,728,"['Bars', 'American (New)', 'Coffee & Tea']"
19624,NewYork,Saint Tuesday,4.0,30,"['Lounges', 'Speakeasies', 'Cocktail Bars']"
19625,NewYork,Harlem Nights Bar,4.0,185,"['Bars', 'Music Venues']"


In [6]:
# Explode the 'categories' column
city_encoded_data['categories'] = city_encoded_data['categories'].apply(eval)
city_encoded_data = city_encoded_data.explode('categories')
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,Festivals
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts
0,Denver,Jackalope Arts,5.0,8,Local Flavor
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning
...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues
19626,NewYork,Cardiff Giant,4.0,56,Bars
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits"


In [7]:
pathname = '../G4_Project4/Resources/df_Extractions/city_encoded_data.csv'
city_encoded_data.to_csv(pathname, index=False)

In [56]:
# Categories - labelEncoder
# Create a label encoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'categories' column
city_encoded_data['categories_encoded'] = label_encoder.fit_transform(city_encoded_data['categories'])

city_encoded_data

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Festivals,220
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts,38
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces,605
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning,416
...,...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars,58
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues,384
19626,NewYork,Cardiff Giant,4.0,56,Bars,58
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits",70


In [67]:
# Create the city_category_predict_df
city_category_predict_df = city_encoded_data[['city', 'name', 'rating', 'review_count', 'categories','categories_encoded']]
city_category_predict_df

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Festivals,220
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts,38
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces,605
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning,416
...,...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars,58
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues,384
19626,NewYork,Cardiff Giant,4.0,56,Bars,58
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits",70


## Random Forest ML Model

### Model Creation

In [89]:
# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model

RandomForestClassifier(n_estimators=10000, random_state=42)

### Train & Test our model

In [90]:
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.5381600249950294

In [91]:
model.fit(X_test, y_test)

RandomForestClassifier(n_estimators=10000, random_state=42)

In [92]:
model.score(X_test, y_test)

0.5457850488525335

### Predict the city

In [81]:
# Generate a random sample of 25 categories
sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 25)
sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})
category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

In [82]:
# Print the categories being used to predict the city
print("Categories Used:")
for category in category_names:
    print(category)

Categories Used:
Breweries
Pizza
Beer Bar
Coffee & Tea
Food Trucks
Burgers
American (New)
Sandwiches
Cocktail Bars
Steakhouses
Breakfast & Brunch
Tapas/Small Plates
Performing Arts
Trainers
Ramen
Seafood
Cafes
Jewelry
Thai
Massage
Cheesesteaks
Psychics
Spiritual Shop
Speakeasies


In [83]:
# Predict the city based on the random group of 10 categories
prediction = model.predict(sample_categories_encoded)
predicted_city = prediction[0]
predicted_city

'NewYork'

### List top 3 experiences for each category within predicted city 

In [84]:
# Set to keep track of predicted cities and their activities
predicted_data = []

# Set to keep track of unique experiences
unique_experiences = set()

# Get the top activities for each sample category
for category in sample_categories:
    # Convert category back to its name
    category_name = city_category_predict_df.loc[city_category_predict_df['categories_encoded'] == category, 'categories'].iloc[0]

    filtered_activities = city_category_predict_df[(city_category_predict_df['city'] == predicted_city) & (city_category_predict_df['categories_encoded'] == category)]
    sorted_activities = filtered_activities.sort_values(['rating', 'review_count'], ascending=[False, False])
    top_activities = sorted_activities[['name', 'rating', 'review_count']].head(3)

    # Create a new activities_data list for each category iteration
    activities_data = []

    for _, activity in top_activities.iterrows():
        activity_name = activity['name']
        rating = activity['rating']
        review_count = activity['review_count']
                
        # Create a unique experience key
        experience_key = (category_name, predicted_city, activity_name)

        # Check if the experience is unique
        if experience_key not in unique_experiences:
            # Add the experience to the set of unique experiences
            unique_experiences.add(experience_key)

            # Append the predicted activity to the list
            activities_data.append({
                'Sample Category': category_name,
                'Predicted City': predicted_city,
                'Experience Name': activity_name,
                'Rating': rating,
                'Review Count': review_count
            })

    # Append the activities_data to the predicted_data list for each category
    predicted_data.extend(activities_data)

# Create a DataFrame from the predicted data
predicted_df = pd.DataFrame(predicted_data)

# Print the final predicted data DataFrame
print("Predicted user experiences based on predicted city:")
predicted_df

Predicted user experiences based on predicted city:


,Sample Category,Predicted City,Experience Name,Rating,Review Count
0,Thai,NewYork,ME Chicken Rice,5.0,107
1,Thai,NewYork,Glin Thai Bistro,5.0,69
2,Thai,NewYork,Mr Khao Man Gai,5.0,39
3,Seafood,NewYork,Crab Shack Seafood Boil,5.0,20
4,Seafood,NewYork,Thai Food Near Me,5.0,16
5,Seafood,NewYork,The Press Club Grill,5.0,15
6,Cheesesteaks,NewYork,Olde City Cheesesteaks & Brew,5.0,13
7,Beer Bar,NewYork,Crown Alley,5.0,166
8,Cocktail Bars,NewYork,Crown Alley,5.0,166
9,Ramen,NewYork,Waikiki Poke & Ramen,5.0,18


In [76]:
# Write the DataFrame to a CSV file
pathname = '../G4_Project4/Resources/Insights/predicted_df.csv'
predicted_df.to_csv(pathname, index=False)